In [ ]:
import numpy as np 
from scipy.stats import norm
from scipy.optimize import brentq

In [ ]:
class OptionsMathSkew(OptionsMathHelpers):

    
    def __init__(self):
        # constructor 
        pass

  
    # -----------------------
    # SABR volatility skew formulas
    # ----------------------- 
    def sabr_skew(self, u_fv=-100.0, k=-100.0, t=-100.0, v_atm=-100.0, 
                  beta=-100.0, rho_corr=-100.0, nu_vol_of_vol=-100.0):  
        # formula solves for alpha; alpha is not a given

        u_fv, k, t, v_atm, beta, rho_corr, nu_vol_of_vol = self.to_arrays(u_fv, k, t, v_atm, 
                                                                          beta, rho_corr, nu_vol_of_vol)  
       
        # Initialize vols array
        vols = np.empty_like(u_fv, dtype=float)
        z_over_x_z = np.empty_likeu_fv, dtype=float)

        # Define ATM and non-ATM masks
        atm_mask     = np.isclose(u_fv, k, rtol=1e-12, atol=0.0)
        non_atm_mask = ~atm_mask
        
        # Calc variables for repeated use
        one_minus_beta  = 1.0 - beta
        one_minus_beta_over_two = one_minus_beta / 2.0
        one_minus_beta_squared = one_minus_beta * one_minus_beta
        fwd_strike_beta = (u_fv * k) ** one_minus_beta_half

        # Calc third correction term (need to solve for alpha before computing first two correction terms)
        corr_term3_num = (2.0 - (3.0 * rho_corr * rho_corr)) * nu_vol_of_vol * nu_vol_of_vol
        corr_term3     = corr_term3_num / 24

                # Assign z_over_x_z
        z_over_chi[atm_mask]     = 1.0
        z_over_chi[non_atm_mask] = z[non_atm_mask] / chi[non_atm_mask]
                
        # Compute vols
        vols = common_term * z_over_chi
        
        return vols


    def sabr_objective(self, u_fv=-100.0, k=-100.0, t=-100.0, v=-100.0, mkt_v=-100.0,
                       alpha=-100.0, beta=-100.0, rho_corr=-100.0, nu_vol_of_vol=-100.0):
        
        test_v = self.sabr_skew(u_fv=u_fv, k=k, t=t, v_atm=-100.0, alpha=alpha, 
                                 beta=beta, rho_corr=rho_corr, nu_vol_of_vol=nu_vol_of_vols)       
        error = test_v - mkt_v
#weight error by vega        
        return np.sum(error * error)
    #result = minimize(objective, initial_guess, args=(u, k, mkt_vols, v_atm),
    #                  bounds=bounds, method='L-BFGS-B')

    
    # -----------------------
    # Wow-Alpha volatility skew formulas (from SIG - old)
    # ----------------------- 
    def wow_alpha_vol_formula(self, u_fv=-100.0, k=-100.0, 
                              v_atm=-100.0, wow=-100.0, convexity=-100.0, **kwargs):

                log_moneyness = np.log(u_fv / k)      # this is the alpha of "wow alpha"
        
        hi_vol = v_atm * (1 + wow - (log_moneyness * convexity))
        lo_vol = v_atm * (1 - wow - (log_moneyness * convexity))
  
        return hi_vol, lo_vol

    
    def wow_alpha_value_formula(self, c_or_p='', u_fv=-100.0, k=-100.0, t=-100.0, r=0.0,
                                v_atm=-100.0, wow=-100.0, convexity=-100.0, **kwargs):
        
        hi_vol, lo_vol = self.wow_alpha_vol_formula(u_fv=u_fv, k=k, 
                                                    v_atm=v_atm, wow=wow, convexity=convexity)
        
        hi_vol_value = self.bs_option_value_formula(c_or_p=c_or_p, u=u_fv, k=k, t=t, r=r, v=hi_vol)
        lo_vol_value = self.bs_option_value_formula(c_or_p=c_or_p, u=u_fv, k=k, t=t, r=r, v=lo_vol)
        
        wow_alpha_value = (hi_vol_value + lo_vol_value) / 2
        
        return wow_alpha_value


    def wow_alpha_objective():
        pass

        